# Hydro Generation Validation for Zambia

In this notebook, we will focus on the hydro validation of the power plants within the workflow, using data obtained from Zesco. The validation process is divided into two parts: the first part involves validating the hydro energy produced by hydro dams, and the second part involves validating the hydro energy produced from run-of-river sources


## Preparation

In [ ]:
import logging
import os

import pypsa
import yaml
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy as sp
import networkx as nx

# plotting stuff
import matplotlib.pyplot as plt

plt.style.use("bmh")
import seaborn as sns  ###
import cartopy.crs as ccrs

sns.set(style="darkgrid")

from scipy.sparse import csgraph
from itertools import product

from shapely.geometry import Point, LineString
import shapely, shapely.prepared, shapely.wkt

logger = logging.getLogger(__name__)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 70)

# change current directory to parent folder
import os
import sys



# plotting settings
max_node_size = 1.0  # maximum size of a node for plotting purposes [used in plots]

## Loading the solved network

In [3]:
n=pypsa.Network("C:\\Users\\user\\Desktop\\pypsa-earth\\results\\networks\\elec_s_10_ec_lcopt_Co2L-12H.nc")



INFO:pypsa.io:Imported network elec_s_10_ec_lcopt_Co2L-12H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


## Calculating total hydro energy production by powerplants

### Total energy production by dams [TWh]

In [8]:
hydroenergy_by_dams = n.storage_units_t.p.mean().sum() / 1000000 * 8760
hydroenergy_by_dams

7.2123960402408

### Total energy production by run-of-river [TWh]

In [15]:
ror_gen_index = n.generators[n.generators.carrier == "ror"].index
hydroenergy_by_ror = n.generators_t.p[ror_gen_index].mean().sum() / 1000000 * 8760
hydroenergy_by_ror


0.7738610148

In [11]:
n.storage_units_t.keys()

dict_keys(['p_min_pu', 'p_max_pu', 'p_set', 'q_set', 'marginal_cost', 'marginal_cost_quadratic', 'state_of_charge_set', 'efficiency_store', 'efficiency_dispatch', 'standing_loss', 'inflow', 'p', 'p_dispatch', 'p_store', 'q', 'state_of_charge', 'spill', 'mu_upper', 'mu_lower', 'mu_state_of_charge_set', 'mu_energy_balance'])

### Total hydropower energy production [TWh]

In [18]:
tot_hydroenergy = float(hydroenergy_by_dams) + float(hydroenergy_by_ror)


In [19]:
tot_hydroenergy

7.9862570550408005

## Validation of the workflow results with respect to ZESCO data

### The data obtained from the workflow was compared with the data from zesco

| Source | Hydro energy from dams | Hydro energy from ror sources | Total|
|--------|----------------|---------------|---------------|
|Pypsa Earth | 7.21 TWh | 0.774 TWh | 7.986 TWh|
| ZESCO | 9.4063 TWh | 1.0096 TWh | 10.419 TWh|
